# Prototype Bemchmark
For each record to be scored, the "county" specific model is read from external storage.  No model caching in memory.

* `model_load_time` (msec) is the time required to read a model file from external storage and deserialize the model object.
* `model_score_time` (msec) is the time to score one record.

## Metrics reported in this notebook are from synthetic data and **have not** been calibrated to representative dataset or model sizes.

## Notebook run-time enviornment
* **Hardware:** MacBook Pro Intel(2019), 16GB RAM, 1TB SSD drive
* **OS:** MacOS 11.6.1
* **Docker:** Docker for Desktop 4.2.0 (Mac)
* **Docker Image:** Base image: `jupyter/datascience-notebook:lab-3.2.5` with ONNX packages added

In [1]:
import glob
import os
import shutil
import pandas as pd
import numpy as np
import onnxruntime as rt
import pickle

## Setup for tests

In [2]:
# required to allow for import of project speccific utility functions
os.chdir('..')

In [3]:
# import project specific utiity functions
from utils.utils import BenchmarkDriver, load_config, actualsize_mb

In [4]:
# get configuration parameters
config = load_config('./config_testbed_500.yaml')
config

{'data_dir': '/Users/jim/Desktop/onnx_sandbox/data/testbed',
 'models_dir': '/Users/jim/Desktop/onnx_sandbox/models/testbed500',
 'number_records': 100000,
 'number_features': 20,
 'number_informative': 14,
 'number_trees': 500,
 'fraction_for_test': 0.2,
 'number_counties': 20,
 'random_seed': 123}

In [5]:
TEST_DATA = os.path.join(config['data_dir'], 'test.parquet')
PERFORMANCE_DATA_DIR = os.path.join(config['data_dir'],'performance500')
MODELS_DIR = config['models_dir']


In [6]:
# setup directory to collect performance data
shutil.rmtree(PERFORMANCE_DATA_DIR, ignore_errors=True)
os.makedirs(PERFORMANCE_DATA_DIR, exist_ok=True)

## Test Design

### Model Training
In an offline process, trained 20 Random Forest (RF) models using synthetic data generated by `sklearn.datasets.make_regression()` method.  These data sets have 20 explanatory variables (`X_00` to `X_19`) with a target variable called 'y'.  With the exception of setting `random_state` and `n_estimators=500` parameter to a known value, all other hyper-parameters were allowed to take their default value.

In [7]:
pd.read_parquet(os.path.join(TEST_DATA)).head()

,county,X_00,X_01,X_02,X_03,X_04,X_05,X_06,X_07,X_08,...,X_11,X_12,X_13,X_14,X_15,X_16,X_17,X_18,X_19,y
0,cnty0017,0.520762,-0.658164,0.234148,-1.949334,-1.042321,0.206521,1.468614,0.402485,-0.329271,...,1.148844,0.637367,-1.228588,-0.052683,-1.478516,-2.905790,-0.816058,-0.353458,-1.478990,-656.045166
1,cnty0002,-1.719247,0.145194,0.682192,1.195691,-0.298329,0.274303,1.889344,-1.349296,-0.027038,...,1.903116,0.869600,0.350681,0.064682,-0.017302,0.296412,-0.155503,-0.430120,0.386999,174.201584
2,cnty0005,-0.572314,-0.771446,-0.112503,0.274605,0.546655,0.919573,-1.015905,0.091013,-0.798503,...,0.244685,0.077570,0.540681,0.368747,-0.538600,0.481323,0.122542,1.925718,1.102175,79.194344
3,cnty0000,0.558090,-0.571827,1.589408,0.340502,-1.313514,0.500834,-0.114611,0.248661,-1.692944,...,-0.037938,-0.481162,1.001708,-1.416378,-1.177559,-0.419277,0.349913,-1.670872,0.788682,25.678637
4,cnty0018,-0.383470,-0.381675,1.018075,0.127476,-0.138571,-0.958881,0.165615,-0.385700,0.624114,...,1.220423,-0.427295,-0.472333,-0.883766,0.192109,0.349223,0.846627,1.677837,0.582563,58.992062


For each Random Forest model, saved two model files: sklearn (.pkl file extension) and onnx (.onnx file extension).  These files are indentified by the `county` data used to generate the RF model.  Here is a sample of saved model files.

In [8]:
os.listdir(MODELS_DIR)[:10]

['cnty0000.onnx',
 'cnty0000.pkl',
 'cnty0001.onnx',
 'cnty0001.pkl',
 'cnty0002.onnx',
 'cnty0002.pkl',
 'cnty0003.onnx',
 'cnty0003.pkl',
 'cnty0004.onnx',
 'cnty0004.pkl']

### Model Scoring
For this test selected 100 random records from the test data.  For each record performed the following:
```
# Process test batch
while there are input records:
    Read one record
    Based on 'county' value, load the RF model for that 'county'  
    Record time to load and make model useable
    Score the record
    Record process memory RSS value
    Record time to score the record

# record collected metrics
Write collected run-time metrics to an external file.
```
No explicit caching of model objects were done in this test.

## Analysis of RF Tree Structure


In [9]:
# collect data on RF tree structure
tree_metrics = []
rf_models = glob.glob(os.path.join(MODELS_DIR, '*.pkl'))
for model in rf_models:
    # get file sizes
    fp_parts = os.path.splitext(model)
    metrics = {'model': fp_parts[0].split('/')[-1]}
    metrics['sklearn_file_size_mb'] = os.path.getsize(model) / (1024 * 1024)
    metrics['onnx_file_size_mb'] = os.path.getsize(fp_parts[0] + '.onnx') / (1024 * 1024)
    
    # extract tree structure
    with open(model, 'rb') as f:
        rf = pickle.load(f)
    metrics['number_of_trees'] = len(rf.estimators_)
    tree_depth = [tree.tree_.max_depth for tree in rf.estimators_]
    metrics['tree_min_depth'] = np.min(tree_depth)
    metrics['tree_max_depth'] = np.max(tree_depth)
    metrics['tree_mean_depth'] = np.mean(tree_depth)
    
    del rf
        
    # collect metrics
    tree_metrics.append(metrics)

In [10]:
# overview of tree structure
tree_metrics_df = pd.DataFrame(tree_metrics)
tree_metrics_df

,model,sklearn_file_size_mb,onnx_file_size_mb,number_of_trees,tree_min_depth,tree_max_depth,tree_mean_depth
0,cnty0000,150.833240,94.321342,500,19,28,22.212
1,cnty0001,152.600574,95.429460,500,19,29,22.202
2,cnty0002,154.302966,96.496597,500,19,27,22.236
3,cnty0003,153.341418,95.893731,500,19,29,22.302
4,cnty0004,156.546130,97.902418,500,19,28,22.580
5,cnty0005,157.166125,98.290984,500,19,27,22.380
6,cnty0006,157.051257,98.219077,500,19,27,22.190
7,cnty0007,157.744006,98.653157,500,19,28,22.182
8,cnty0008,154.263049,96.471406,500,19,28,22.062
9,cnty0009,156.009631,97.566100,500,20,30,22.500


In [11]:
tree_metrics_of_interest = ['sklearn_file_size_mb', 'onnx_file_size_mb',
                           'number_of_trees', 'tree_min_depth', 'tree_mean_depth', 'tree_max_depth']
tree_metrics_df[tree_metrics_of_interest].describe().T

,count,mean,std,min,25%,50%,75%,max
sklearn_file_size_mb,20.0,154.422046,2.139231,150.833240,152.825336,154.090808,156.315234,157.744006
onnx_file_size_mb,20.0,96.571051,1.340894,94.321342,95.570291,96.363454,97.757710,98.653157
number_of_trees,20.0,500.000000,0.000000,500.000000,500.000000,500.000000,500.000000,500.000000
tree_min_depth,20.0,19.200000,0.410391,19.000000,19.000000,19.000000,19.000000,20.000000
tree_mean_depth,20.0,22.298600,0.140032,22.062000,22.209500,22.262000,22.381500,22.580000
tree_max_depth,20.0,28.000000,0.973329,27.000000,27.000000,28.000000,28.250000,30.000000


## Model Scoring Test

### Get data for test

In [12]:
test_df = pd.read_parquet(TEST_DATA)
test_df = test_df.sample(n=100, random_state=config['random_seed'])
test_df.reset_index(inplace=True)
test_df.shape

(100, 23)

In [13]:
test_df[['index','county', 'y']].head(10)

,index,county,y
0,12136,cnty0009,21.728182
1,16812,cnty0018,-191.266129
2,10072,cnty0019,18.938940
3,5850,cnty0015,57.200191
4,4320,cnty0008,-154.245728
5,4429,cnty0019,-46.112324
6,11744,cnty0016,-48.300835
7,12106,cnty0008,247.827332
8,4230,cnty0009,-219.337921
9,16858,cnty0003,-271.940826


In [14]:
metrics_of_interest = ['model_process_rss_mb', 'model_load_time_ms', 'model_score_time_ms', 'model_prediction_time_ms']

### sklearn Model Scoring

In [15]:
%%time
sklearn_driver = BenchmarkDriver(
    model_type='sklearn',
    models_dir=MODELS_DIR,
    performance_fp=os.path.join(PERFORMANCE_DATA_DIR, 'sklearn_benchmark.csv'),
    test_scenario='county-level'
)

# iterate over each row and collect run-time performance statistics
for idx, row in test_df.iterrows():
    one_record = pd.DataFrame(row).T
    sklearn_driver.score_one_record(row['county'], row['index'],
            one_record.drop(['index', 'county', 'y'], axis='columns'))
    


CPU times: user 9.8 s, sys: 7.52 s, total: 17.3 s
Wall time: 16 s


In [16]:
# display collected performance metrics
sklearn_metrics_df = pd.read_csv(os.path.join(PERFORMANCE_DATA_DIR, 'sklearn_benchmark.csv'))
sklearn_metrics_df['model_prediction_time_ms'] = sklearn_metrics_df['model_load_time_ms'] + sklearn_metrics_df['model_score_time_ms']
sklearn_metrics_df.head(10)

,county_id,record_id,test_scenario,model_load_time_ms,model_score_time_ms,model_process_rss_mb,predicted_score,model_prediction_time_ms
0,cnty0009,12136,county-level,58.0551,104.9817,529.324219,50.785638,163.0368
1,cnty0018,16812,county-level,51.3072,101.6873,521.363281,-151.409153,152.9945
2,cnty0019,10072,county-level,54.7442,105.8678,525.253906,-96.466683,160.6120
3,cnty0015,5850,county-level,51.5631,95.3381,525.234375,95.595544,146.9012
4,cnty0008,4320,county-level,52.5010,96.5703,522.269531,-90.327317,149.0713
5,cnty0019,4429,county-level,52.5943,96.6763,525.343750,-40.367087,149.2706
6,cnty0016,11744,county-level,51.4295,109.3827,526.441406,-58.800257,160.8122
7,cnty0008,12106,county-level,51.4661,99.4886,522.265625,20.205901,150.9547
8,cnty0009,4230,county-level,54.9318,94.5876,523.425781,-88.128245,149.5194
9,cnty0003,16858,county-level,50.7944,102.0625,518.742188,-208.371904,152.8569


In [17]:
sklearn_metrics_df[metrics_of_interest].describe(percentiles=[.25, .5, .75, .9]).T

,count,mean,std,min,25%,50%,75%,90%,max
model_process_rss_mb,100.0,522.141484,2.850434,515.878906,520.140625,523.462891,524.035156,525.091406,529.324219
model_load_time_ms,100.0,53.235108,2.258203,49.490000,51.649000,52.841950,54.419200,55.931180,63.092700
model_score_time_ms,100.0,99.975537,5.985136,78.009400,96.649800,99.731050,103.473600,107.563680,116.010600
model_prediction_time_ms,100.0,153.210645,6.591434,130.222200,149.214250,152.744550,156.916300,160.829980,171.719800


### onnx scoring test

In [ ]:
%%time
onnx_driver = BenchmarkDriver(
    model_type='onnx',
    models_dir=MODELS_DIR,
    performance_fp=os.path.join(PERFORMANCE_DATA_DIR, 'onnx_benchmark.csv'),
    test_scenario='county-level'
)

# iterate over each row and collect run-time performance statistics
for idx, row in test_df.iterrows():
    one_record = pd.DataFrame(row).T
    onnx_driver.score_one_record(row['county'], row['index'],
            one_record.drop(['index', 'county', 'y'], axis='columns').astype(np.float32).to_numpy())


In [ ]:
# display collected performance metrics
onnx_metrics_df = pd.read_csv(os.path.join(PERFORMANCE_DATA_DIR, 'onnx_benchmark.csv'))
onnx_metrics_df['model_prediction_time_ms'] = onnx_metrics_df['model_load_time_ms'] + onnx_metrics_df['model_score_time_ms']
onnx_metrics_df.head(10)

In [ ]:
onnx_metrics_df[metrics_of_interest].describe(percentiles=[.25, .5, .75, .9]).T

### Differences in sklearn vs onnx predictions

In [ ]:
differences = np.abs(sklearn_metrics_df['predicted_score'] - onnx_metrics_df['predicted_score'])
differences.describe()